This notebook seeks to explore the gender diversity of the different apache projects & the process

In [1]:
!pwd
!ls /hadoop/spark/tmp/
!cd /hadoop/spark/tmp/spark-*/userFiles-*/
!pwd
!rm lazy_helpers.py
!wget https://raw.githubusercontent.com/holdenk/diversity-analytics/master/lazy_helpers.py

/
blockmgr-1f117d91-c674-45a3-b47c-ca7284bcc685
blockmgr-6ed4b2da-f064-417b-b9d1-0778183fb623
spark-6207646a-f0a7-42ce-8f11-3babed4e79aa
spark-b9a4f48d-cf45-4c59-a802-97f0003ebd85
/
--2018-07-13 00:34:33--  https://raw.githubusercontent.com/holdenk/diversity-analytics/master/lazy_helpers.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1397 (1.4K) [text/plain]
Saving to: ‘lazy_helpers.py’

lazy_helpers.py     100%[=====================>]   1.36K  --.-KB/s   in 0s     

2018-07-13 00:34:33 (40.3 MB/s) - ‘lazy_helpers.py’ saved [1397/1397]



In [2]:
#Hack to update sparklingml on a running cluster
memory_status_count = sc._jsc.sc().getExecutorMemoryStatus().size()
estimated_executors = max(sc.defaultParallelism, memory_status_count)
rdd = sc.parallelize(range(estimated_executors))
def do_update(x):
    import os
    return str(os.popen("cd /sparklingml && git pull && git log -n 5").read())
result = rdd.map(do_update)
result.collect()

['',
 'Already up-to-date.\ncommit 621a5736855a69a9684b2c926f778c2da3a61c95\nAuthor: Holden Karau <holden@pigscanfly.ca>\nDate:   Thu Jul 12 17:11:53 2018 -0700\n\n    Works in Python 2 and 3 now\n\ncommit 91ff6b839c1bedf6b376e496386c63e35e85a330\nAuthor: Holden Karau <holden@pigscanfly.ca>\nDate:   Thu Jul 12 17:09:38 2018 -0700\n\n    Working in Python 3\n\ncommit 358458dfa152b8255d703a0e91b4f39b6012f241\nAuthor: Holden Karau <holden@pigscanfly.ca>\nDate:   Thu Jul 12 16:51:16 2018 -0700\n\n    Put pandas back in with a noqa skip for flake8 since we use it in the doc tests.\n\ncommit 388118867fb818459184b67a07203e41111ad956\nAuthor: Holden Karau <holden@pigscanfly.ca>\nDate:   Thu Jul 12 16:26:03 2018 -0700\n\n    Remove home/build from the cache\n\ncommit 81ea25aa31173738d35d3a4e98f9648042c3ac7c\nAuthor: Holden Karau <holden@pigscanfly.ca>\nDate:   Thu Jul 12 16:22:52 2018 -0700\n\n    Fix flake8s\n']

In [3]:
import os
os.environ['PATH'] = os.environ['PATH'] + ":/usr/lib/chromium/"

In [4]:
from pyspark import *
from pyspark.sql import *
from pyspark.sql.functions import concat, collect_set, explode, from_json, format_string
from pyspark.sql import functions as F
from pyspark.sql.session import *
from pyspark.sql.types import *

import json
import os
import meetup.api
from copy import copy
import time
import logging
import subprocess

API key configuration

In [5]:
exec(open("./secrets.literal").read()) # Load gh_api_token & meetup_key
gh_user = "holdenk"
fs_prefix = "gs://boo-stuff/"

Less secret configuration

In [6]:
max_meetup_events = 800

In [7]:
session = (SparkSession.builder
           .appName("whatCanWeLearnFromTheSixties")
           .config("spark.executor.instances", "40")
           .config("spark.driver.memoryOverhead", "0.25")
           .config("spark.executor.memory", "16g")
           .config("spark.dynamicAllocation.enabled", "false")
           .config("spark.ui.enabled", "true")
          ).getOrCreate()
sc = session.sparkContext

In [8]:
# In _theory_ in preview dataproc Spark UI is force disabled but history fills the gap, except history server isn't started by default :(
sc.getConf().get("spark.yarn.historyServer.address")

'holden-magic-m:18080'

The first thing we want to get is the committers and PMC members, this information is stored in LDAP but also available in JSON. Eventually we will want to enrich this with mailing list information

In [9]:
def loadFlatJsonFile(path, explodeKey, schema=None):
    """Load a flat multi-line json file and convert into Spark & explode"""
    rdd = sc.wholeTextFiles(path).values().setName("Input file {}".format(path))
    df = (session.read.schema(schema)
            .json(rdd))
    return df.select(explode(explodeKey))

In [10]:
apache_people_schema = StructType([StructField("lastCreateTimestamp", StringType()),
                     StructField("people",
                                 MapType(StringType(), 
                                         StructType([StructField('name', StringType()),
                                                     StructField('key_fingerprints', ArrayType(StringType())),
                                                     StructField('urls', ArrayType(StringType())),
                                                    ]))
                                )])
apache_poeple_df_file = "{0}{1}".format(fs_prefix, "http_data_sources/public_ldap_people.json") # http://people.apache.org/public/public_ldap_people.json
apache_people_df = loadFlatJsonFile(path=apache_poeple_df_file, 
                                 explodeKey="people", schema=apache_people_schema)
apache_people_df = apache_people_df.select(apache_people_df.key.alias("username"), apache_people_df.value.alias("extra")).repartition(100).persist().alias("apache_people")
apache_people_df.alias("Apache Committers")

DataFrame[username: string, extra: struct<name:string,key_fingerprints:array<string>,urls:array<string>>]

In [11]:
sc.addFile("lazy_helpers.py")

In [12]:
# Construct a lazy urllib3 pool
from lazy_helpers import *
    
bcast_pool = sc.broadcast(LazyPool)
bcast_pool.value

lazy_helpers.LazyPool

In [13]:
def project_on_github(project):
    """Returns if a project is on github"""
    import urllib3
    http = bcast_pool.value.get()
    r = http.request('GET', "https://github.com/apache/{0}".format(project))
    return r.status == 200
session.catalog.registerFunction("on_github", project_on_github, BooleanType())
# Except I'm a bad person so....
from pyspark.sql.catalog import UserDefinedFunction
project_on_github_udf = UserDefinedFunction(project_on_github, BooleanType(), "on_github")
session.catalog._jsparkSession.udf().registerPython("on_github", project_on_github_udf._judf)

In [14]:
apache_committees_schema = StructType([StructField("lastCreateTimestamp", StringType()),
                     StructField("committees",
                                 MapType(StringType(), StructType([StructField('roster', ArrayType(StringType())),
                                                                  StructField('modifyTimestamp', StringType()),
                                                                  StructField('createTimestamp', StringType())
                                                                  ])))])
apache_committees_df_file = "{0}{1}".format(fs_prefix, "http_data_sources/public_ldap_committees.json") # http://people.apache.org/public/public_ldap_committees.json
apache_committees_df = loadFlatJsonFile(path=apache_committees_df_file,
                                 explodeKey="committees", schema=apache_committees_schema)
apache_committees_on_github_df = apache_committees_df.filter(project_on_github_udf(apache_committees_df.key))
apache_committees_on_github_df.persist(StorageLevel.MEMORY_AND_DISK)
committee_names_df = apache_committees_on_github_df.select(apache_committees_df.key.alias("project")).alias("apache_committees").repartition(200)
committee_names_df.persist(StorageLevel.MEMORY_AND_DISK)
committee_names_df.alias("Apache Committee Names")
committee_names_df.count()

163

In [15]:
project_to_user_df = apache_committees_on_github_df.select(
    apache_committees_on_github_df.key.alias("project"),
    explode(apache_committees_on_github_df.value.roster).alias("username"))


user_to_project_df = project_to_user_df.groupBy(project_to_user_df.username).agg(
    collect_set(project_to_user_df.project).alias("projects"))
apache_people_df = apache_people_df.join(user_to_project_df, on="username")
apache_people_df.alias("Apache People joined with projects")

DataFrame[username: string, extra: struct<name:string,key_fingerprints:array<string>,urls:array<string>>, projects: array<string>]

In [16]:
apache_people_df.take(1)

[Row(username='macdonst', extra=Row(name='Simon MacDonald', key_fingerprints=None, urls=None), projects=['cordova'])]

Attempt to fetch relevant past & present meetups for each project - idea based on the listing at https://www.apache.org/events/meetups.html but different code

We want to do a non-blocking count to materialize the meetup RDD because this is slow

In [17]:
# Some async helpers, in Scala we would use AsyncRDDActions but its not currently available in Python
# Support is being considered in https://issues.apache.org/jira/browse/SPARK-20347
def non_blocking_rdd_count(rdd):
    import threading
    def count_magic():
        rdd.count()
    thread = threading.Thread(target=count_magic)
    thread.start()

def non_blocking_rdd_save(rdd, target):
    import threading
    def save_panda():
        rdd.saveAsPickleFile(target)
    thread = threading.Thread(target=save_panda)
    thread.start()

def non_blocking_df_save(df, target):
    import threading
    def save_panda():
        df.write.mode("overwrite").save(target)
    thread = threading.Thread(target=save_panda)
    thread.start()

def non_blocking_df_save_csv(df, target):
    import threading
    def save_panda():
        df.write.format("csv").mode("overwrite").option("header", "true").save(target)
    thread = threading.Thread(target=save_panda)
    thread.start()

def non_blocking_df_save_or_load(df, target):
    fs = sc._jvm.org.apache.hadoop.fs.FileSystem.get(sc._jvm.java.net.URI(fs_prefix), sc._jsc.hadoopConfiguration())
    success_files = ["{0}/SUCCESS.txt", "{0}/_SUCCESS"]
    if any(fs.exists(sc._jvm.org.apache.hadoop.fs.Path(t.format(target))) for t in success_files):
        print("Reusing")
        return session.read.load(target).persist()
    else:
        print("Saving")
        non_blocking_df_save(df, target)
        return df

def non_blocking_df_save_or_load_csv(df, target):
    fs = sc._jvm.org.apache.hadoop.fs.FileSystem.get(sc._jvm.java.net.URI(fs_prefix), sc._jsc.hadoopConfiguration())
    success_files = ["{0}/SUCCESS.txt", "{0}/_SUCCESS"]
    if any(fs.exists(sc._jvm.org.apache.hadoop.fs.Path(t.format(target))) for t in success_files):
        print("Reusing")
        return session.read.format("csv").option("header", "true").option("inferSchema", "true").load(target).persist()
    else:
        print("Saving")
        non_blocking_df_save_csv(df, target)
        return df

In [18]:
logger = logging.getLogger()
logger.setLevel("WARN")
# For now, this is an avenue of future exploration, AKA holden's doesn't want her meetup API keys banned
def lookup_relevant_meetup(project_name, max_meetup_events=0):
    """Lookup relevant meetups for a specific project."""
    import logging
    import time
    import meetup.api
    logger = logging.getLogger()
    meetup_delay = 30
    meetup_reset_delay = 3600 # 1 hour
    standard_keys = {"text_format": "plain", "trending": "desc=true", "and_text": "true", "city": "san francisco", "country": "usa", "text": "apache " + project_name, "radius": 10000}
    results = {"upcoming": [], "past": []}
    for status in ["upcoming", "past"]:
        keys = copy(standard_keys)
        keys["status"] = status
        count = 200
        base = 0
        while (count == 200 and (max_meetup_events == 0 or base < max_meetup_events)):
            logging.debug("Fetch {0} meetups for {1} on base {2}".format(status, project_name, base))
            project_name = "spark"
            client = client = meetup.api.Client(meetup_key)
            if base > 0:
                keys["page"] = base
            # Manually sleep for meetup_reset_delay on failure, the meetup-api package retry logic sometimes breaks :(
            response = None
            retry_count = 0
            while response is None and retry_count < 10:
                try:
                    response = client.GetOpenEvents(**keys)
                except:
                    response = None
                    retry_count += 1
                    time.sleep(meetup_reset_delay)
                    try:
                        response = client.GetOpenEvents(**keys)
                    except:
                        response = None
            try:
                count = response.meta['count']
                base = base + count
                results[status].append(response.results)
                time.sleep(meetup_delay)
            except:
                count = 0
    return (project_name, results)

In [19]:
project_meetups_rdd = committee_names_df.repartition(500).rdd.map(lambda x: x.project).map(lambda name: lookup_relevant_meetup(name, max_meetup_events))
project_meetups_rdd.setName("Meetup Data RDD")

Meetup Data RDD PythonRDD[74] at RDD at PythonRDD.scala:48

In [20]:
#project_meetups_rdd.persist(StorageLevel.MEMORY_AND_DISK)
#raw_project_meetups_df = project_meetups_rdd.toDF() 
#raw_project_meetups_df.alias("Project -> meetup dataframe")

In [21]:
#project_meetups_df = non_blocking_df_save(raw_project_meetups_df, "mini_meetup_data")

In [22]:
#project_meetups_df.show()

In [23]:
#project_meetups_df.schema

For the provided projects attempt to lookup their GitHub

In [24]:
def lookup_project_git(org, project):
    """Returns the project github for a specific project. Assumes project is git hosted"""
    return "https://github.com/{0}/{1}.git".format(org, project)
    

In [25]:
def fetch_project_github_data(org, project):
    """Fetch the project github data, note this only gets github issues so likely not super useful"""
    from perceval.backends.core.github import GitHub as perceval_github
    gh_backend = perceval_github(owner=org, repository=project, api_token=gh_api_token)
    # The backend return a generator - which is awesome. However since we want to pull this data into Spark 
    def append_project_info(result):
        """Add the project information to the return from perceval"""
        result["project_name"] = project
        return result

    return list(map(append_project_info, gh_backend.fetch()))

In [26]:
def fetch_project_git_data(org, project):
    from perceval.backends.core.git import Git as perceval_git

    git_uri = lookup_project_git(org, project)
    import tempfile
    import shutil
    tempdir = tempfile.mkdtemp()

    def append_project_info(result):
        """Add the project information to the return from perceval"""
        result["project_name"] = project
        return result

    try:
        git_backend = perceval_git(uri=git_uri, gitpath=tempdir + "/repo")
        return list(map(append_project_info, git_backend.fetch()))
    finally:
        shutil.rmtree(tempdir)

Fetch the git history info using perceval

In [27]:
apache_git_project_data_rdd = committee_names_df.repartition(400).rdd.flatMap(lambda row: fetch_project_git_data("apache", row.project))
jupyter_git_project_data_rdd = sc.parallelize([("jupyter", "notebook"), ("nteract", "nteract")]).flatMap(lambda elem: fetch_project_git_data(elem[0], elem[1]))
git_project_data_rdd = apache_git_project_data_rdd.union(jupyter_git_project_data_rdd)
git_project_data_rdd.setName("Perceival GIT dat")

Perceival GIT dat UnionRDD[86] at union at NativeMethodAccessorImpl.java:0

In [28]:
git_project_data_df_raw = git_project_data_rdd.map(lambda row: Row(**row)).toDF().persist()
git_project_data_df = non_blocking_df_save_or_load(git_project_data_df_raw, "{0}/raw_git_data".format(fs_prefix))

Reusing


In [29]:
git_project_data_df.schema

StructType(List(StructField(backend_name,StringType,true),StructField(backend_version,StringType,true),StructField(category,StringType,true),StructField(data,MapType(StringType,StringType,true),true),StructField(origin,StringType,true),StructField(perceval_version,StringType,true),StructField(project_name,StringType,true),StructField(tag,StringType,true),StructField(timestamp,DoubleType,true),StructField(updated_on,DoubleType,true),StructField(uuid,StringType,true)))

In [30]:
raw_authors_by_project_and_commit_df = git_project_data_df.select("project_name", "data.Author", "data.CommitDate")


In [31]:
raw_authors_by_project_and_commit_df.show()
raw_authors_by_project_and_commit_df.take(1)

+------------+--------------------+--------------------+
|project_name|              Author|          CommitDate|
+------------+--------------------+--------------------+
|  manifoldcf|Grant Ingersoll <...|Mon Jan 11 22:36:...|
|  manifoldcf|Grant Ingersoll <...|Tue Jan 12 15:04:...|
|  manifoldcf|Karl Wright <kwri...|Sat Feb 13 19:06:...|
|  manifoldcf|Karl Wright <kwri...|Sat Feb 13 20:20:...|
|  manifoldcf|Karl Wright <kwri...|Sat Feb 13 21:20:...|
|  manifoldcf|Karl Wright <kwri...|Sat Feb 13 21:42:...|
|  manifoldcf|Karl Wright <kwri...|Sat Feb 13 21:59:...|
|  manifoldcf|Karl Wright <kwri...|Sun Feb 14 01:04:...|
|  manifoldcf|Karl Wright <kwri...|Sun Feb 14 12:08:...|
|  manifoldcf|Karl Wright <kwri...|Sun Feb 14 13:40:...|
|  manifoldcf|Karl Wright <kwri...|Sun Feb 14 14:15:...|
|  manifoldcf|Karl Wright <kwri...|Sun Feb 14 20:37:...|
|  manifoldcf|Karl Wright <kwri...|Mon Feb 15 15:51:...|
|  manifoldcf|Karl Wright <kwri...|Mon Feb 15 20:16:...|
|  manifoldcf|Karl Wright <kwri

[Row(project_name='manifoldcf', Author='Grant Ingersoll <gsingers@apache.org>', CommitDate='Mon Jan 11 22:36:06 2010 +0000')]

In [32]:
@F.pandas_udf(StringType())
def strip_junk(inputSeries):
    """Discard timezone information, who needs that anyways.
    More accurately we don't care about that here since we're looking at a year long window."""
    return inputSeries.apply(lambda x: x.split("+")[0])

@F.pandas_udf(StringType())
def extract_email(inputSeries):
    """Take e-mails of the form Foo Baz<foobaz@baz.com> and turn it into foobaz@baz.com"""
    import re
    def extract_email_record(record):
        try:
            emails = re.findall('<\S+>$', record)
            return emails[0]
        except:
            return record
    
    return inputSeries.apply(extract_email_record)

@F.pandas_udf(StringType())
def extract_name(inputSeries):
    """Take e-mails of the form Foo Baz<foobaz@baz.com> and turn it into the probable name e.g. Foo Baz"""
    import re
    def extract_name_record(record):
        try:
            emails = re.findall('([^<]+)<\S+>$', record)
            return emails[0]
        except:
            return ""
    
    return inputSeries.apply(extract_name_record)

In [33]:
authors_by_project_and_commit_df = raw_authors_by_project_and_commit_df.select(
    "project_name", "Author", extract_email("Author").alias("email"), extract_name("Author").alias("name"),
    F.to_date(strip_junk("CommitDate"), format="EEE MMM d H:mm:ss YYYY ").alias("CommitDate"))

In [34]:
authors_by_project_and_commit_df.schema

StructType(List(StructField(project_name,StringType,true),StructField(Author,StringType,true),StructField(email,StringType,true),StructField(name,StringType,true),StructField(CommitDate,DateType,true)))

In [35]:
raw_distinct_authors_latest_commit = authors_by_project_and_commit_df.groupBy(
    "project_name", "email").agg(
    F.last("Author").alias("Author"), F.max("CommitDate").alias("latest_commit"))
raw_distinct_authors_latest_commit.persist()

DataFrame[project_name: string, email: string, Author: string, latest_commit: date]

In [36]:
distinct_authors_latest_commit = non_blocking_df_save_or_load(
    raw_distinct_authors_latest_commit,
    "{0}distinct_authors_latest_commit_4".format(fs_prefix))

Reusing


In [37]:
@F.pandas_udf(StringType(), functionType=F.PandasUDFType.SCALAR)
def lookup_github_user_by_email(emails):
    
    import time
    from github import Github
    import backoff
    github_client = Github(gh_user, gh_api_token)
    # In theory PyGithub handles backoff but we have multiple instances/machines.
    @backoff.on_exception(backoff.expo, Exception)
    def inner_lookup_github_user_by_email(email):
        """Lookup github user by e-mail address and returns the github username. Returns None if no user or more than 1 user is found."""
        users = github_client.search_users("{0}".format(email))
        def process_result(users):
            if users.totalCount == 1:
                return list(users).pop().login
            else:
                return ""

        return process_result(users)

    return emails.apply(inner_lookup_github_user_by_email)

In [38]:
authors_to_github_username = distinct_authors_latest_commit.withColumn(
    "github_username",
    lookup_github_user_by_email("email"))

In [39]:
@F.pandas_udf(returnType=StringType(), functionType=F.PandasUDFType.SCALAR)
def fetch_github_user_bio(logins):
    from github import Github
    import time
    github_client = Github(gh_user, gh_api_token)
    import backoff

    @backoff.on_exception(backoff.expo, Exception)
    def individual_fetch_github_user_bio(login):
        if login == None or login == "":
            return ""

        result = github_client.get_user(login=login)
        try:
            return result.bio
        except:
            return ""
    
    return logins.apply(individual_fetch_github_user_bio)

In [40]:
authors_to_github_username.persist()
authors_to_github_username_saved = non_blocking_df_save_or_load(
    authors_to_github_username,
    "{0}/authors_to_github-10".format(fs_prefix))

Saving


In [41]:
distinct_authors_latest_commit.schema

StructType(List(StructField(project_name,StringType,true),StructField(email,StringType,true),StructField(Author,StringType,true),StructField(latest_commit,DateType,true)))

In [42]:
authors_to_github_username_saved.schema

StructType(List(StructField(project_name,StringType,true),StructField(email,StringType,true),StructField(Author,StringType,true),StructField(latest_commit,DateType,true),StructField(github_username,StringType,true)))

In [43]:
distinct_authors_with_gh = authors_to_github_username_saved.withColumn(
    "new_unique_id",
    F.when(F.col("github_username") != "",
         F.col("github_username")).otherwise(
        F.col("email")))
                                                                       

In [44]:
authors_grouped_by_id = distinct_authors_with_gh.groupBy("project_name", "new_unique_id").agg(
    collect_set(F.col("email")).alias("emails"),
    F.last(F.col("Author")).alias("Author"),
    F.first("github_username").alias("github_username"),
    F.max("latest_commit").alias("latest_commit"))
    
    

In [45]:
authors_grouped_by_id.schema

StructType(List(StructField(project_name,StringType,true),StructField(new_unique_id,StringType,true),StructField(emails,ArrayType(StringType,true),true),StructField(Author,StringType,true),StructField(github_username,StringType,true),StructField(latest_commit,DateType,true)))

In [46]:
authors_grouped_by_id.persist()
authors_grouped_by_id_saved = non_blocking_df_save_or_load(
    authors_grouped_by_id,
    "{0}/authors_grouped_by_id-3".format(fs_prefix))

Reusing


Lookup info from crunchbase

In [47]:
os.environ['PATH']

'/opt/conda/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/lib/chromium/'

In [48]:
from lazy_helpers import *

bcast_driver = sc.broadcast(LazyDriver)

# TBD if we should see this, see comments on robots.txt in function, also consider overhead of firefox req
def lookup_crunchbase_info(people_and_projects):
    """Lookup a person a crunch base and see what the gender & company is.
    Filter for at least one mention of their projects."""
    # Path hack
    if not "chromium" in os.environ['PATH']:
        os.environ['PATH'] = os.environ['PATH'] + ":/usr/lib/chromium/"
    from bs4 import BeautifulSoup
    import re
    driver = bcast_driver.value.get()
    import time
    import random
    for (username, name, projects, urls) in people_and_projects:
        time.sleep(random.randint(60, 2*60))
        # robots.txt seems to be ok with person for now as of April 4 2018, double check before re-running this
        url = "https://www.crunchbase.com/person/{0}".format(name.replace(" ", "-"))
        try:
            if driver.current_url != url:
                driver.get(url)
            text = driver.page_source
            lower_text = text.lower()
            yield[lower_text]
            if "the quick brown fox jumps over the lazy dog" in lower_text or "pardon our interruption..." in lower_text:
                time.sleep(random.randint(30*60, 2*60*60))
                bcast_driver.value.reset()
            if any(project.lower() in lower_text for project in projects) or any(url.lower in lower_text for url in urls):
                soup = BeautifulSoup(text, "html.parser")
                stats = soup.findAll("div", { "class" : "component--fields-card"})[0]
                # Hacky but I'm lazy
                result = {}
                result["crunchbase-url"] = url
                result["username"] = username
                if "Female" in str(stats):
                    result["gender"] = "Female"
                if "Male" in str(stats):
                    result["gender"] = "Male"
                try:
                    m = re.search("\" title=\"(.+?)\" href=\"\/organization", lower_text)
                    result["company"] = m.group(1)
                except:
                    # No match no foul
                    pass
                yield result
        except Exception as e:
            pass

In [49]:
#result = lookup_crunchbase_info([("holden", "holden karau", ["spark"], ["http://www.holdenkarau.com"])])
#list(result)

Augment the committer info

In [50]:
# We do this as an RDD transformation since the cost of the transformation dominates
relevant_info = apache_people_df.select(
    apache_people_df.username,
    apache_people_df.extra.getField("name").alias("name"),
    apache_people_df.projects,
    apache_people_df.extra.getField("urls").alias("urls"))
crunchbase_info_rdd = relevant_info.rdd.map(lambda row: (row.username, row.name, row.projects, row.urls)).mapPartitions(lookup_crunchbase_info)
crunchbase_info_rdd.persist(StorageLevel.MEMORY_AND_DISK)
schema = StructType([
    StructField("username", StringType()),
    StructField("gender", StringType()),
    StructField("company", StringType()),
    StructField("crunchbase-url", StringType())])
crunchbase_info_df = crunchbase_info_rdd.toDF(schema = schema)
crunchbase_info_df.alias("Crunchbase user information")

DataFrame[username: string, gender: string, company: string, crunchbase-url: string]

In [51]:
crunchbase_info_df = non_blocking_df_save_or_load(
    crunchbase_info_df,
    "{0}crunchbase_out_11".format(fs_prefix))

Saving


In [52]:
#crunchbase_info_df.count()

In [53]:
apache_people_df.count()

2565

Export to Mechnical turk format

In [54]:
def mini_concat_udf(array_strs):
    """Concat the array of strs"""
    if array_strs == None:
        return ""
    else:
        return ' '.join(array_strs)

# Except I'm a bad person so....
from pyspark.sql.catalog import UserDefinedFunction
mini_concat_udf = UserDefinedFunction(mini_concat_udf, StringType(), "mini_concat_udf")
session.catalog._jsparkSession.udf().registerPython("mini_concat_udf", mini_concat_udf._judf)

mini_csv_data_df = apache_people_df.select(
    apache_people_df.username,
    apache_people_df.extra.getField("name").alias("name"),
    mini_concat_udf(apache_people_df.extra.getField("urls")).alias("personal_websites"),
    mini_concat_udf(apache_people_df.projects).alias("projects")
    ).coalesce(1)
mini_csv_data_df = non_blocking_df_save_or_load_csv(mini_csv_data_df, "{0}/apache_people.csv".format(fs_prefix))

Reusing


In [55]:
#crunchbase_info_rdd.collect()

One of the things that is interesting is understanding what the tones of the meetup descriptions & mailing list posts are. We can use https://www.ibm.com/watson/developercloud/tone-analyzer/api/v3/?python#introduction

In [56]:
# TODO: pandas UDF accelerate (but multiple pieces of informaiton returned at the same time)
def lookup_sentiment(document):
    """Looks up the sentiment for a specific document."""
    from nltk.sentiment.vader import SentimentIntensityAnalyzer
    # TODO(holden): Consider broadcast variable?
    sid = SentimentIntensityAnalyzer()
    return sid.polarity_scores(document)

In [57]:
oh_no_you = lookup_sentiment("oh no you didn't girl")

In [58]:
oh_no_you

{'neg': 0.355, 'neu': 0.645, 'pos': 0.0, 'compound': -0.296}

Ok its time to find some mailing list info

In [59]:
mbox_failures = sc.accumulator(0)

def fetch_mbox_ids(project_name):
    """Return the mbox ids"""
    import itertools

    def fetch_mbox_ids_apache_site(box_type):
        """Fetches all of the mbox ids from a given apache project and box type (dev or user)"""
        root_url = "http://mail-archives.apache.org/mod_mbox/{0}-{1}".format(project_name, box_type)
        
        # Fetch the page to parse
        pool = bcast_pool.value.get()
        result = pool.request('GET', root_url)
        
        
        from bs4 import BeautifulSoup
        soup = BeautifulSoup(result.data, "html.parser")
        mbox_ids = set(map(lambda tag: tag.get('id'), soup.findAll("span", { "class" : "links"})))
        return map(lambda box_id: (project_name, box_type, box_id), mbox_ids)
    # We have to return a list here because PySpark doesn't handle generators (TODO: holden)
    return list(itertools.chain.from_iterable(map(fetch_mbox_ids_apache_site, ["dev", "user"])))
        
        
def fetch_and_process_mbox_records(project_name, box_type, mbox_id):
        import tempfile
        import shutil
        from perceval.backends.core.mbox import MBox as perceval_mbox

        def process_mbox_directory(base_url, dir_path):
            mbox_backend = perceval_mbox(base_url, dir_path)
            return mbox_backend.fetch()
        
        def append_project_info(result):
            """Add the project information to the return from perceval"""
            result["project_name"] = project_name
            result["box_type"] = box_type
            result["mbox_id"] = mbox_id
            return result

        # Make a temp directory to hold the mbox files
        tempdir = tempfile.mkdtemp()

        try:
            root_url = "http://mail-archives.apache.org/mod_mbox/{0}-{1}".format(project_name, box_type)
            mbox_url = "{0}/{1}.mbox".format(root_url, mbox_id)
            filename = "{0}/{1}.mbox".format(tempdir, mbox_id)
        
            print("fetching {0}".format(mbox_url))

            pool = bcast_pool.value.get()
            with pool.request('GET', mbox_url, preload_content=False) as r, open(filename, 'wb') as out_file:       
                try:
                    shutil.copyfileobj(r, out_file)
                    return list(map(append_project_info, process_mbox_directory(root_url, tempdir)))
                except:
                    mbox_failures.add(1)
                    return []
        finally:
            shutil.rmtree(tempdir)

In [60]:
def random_key(x):
    import random
    return (random.randint(0, 40000), x)

def de_key(x):
    return x[1]

mailing_list_posts_mbox_ids = committee_names_df.repartition(400).rdd.flatMap(lambda row: fetch_mbox_ids(row.project))
# mbox's can be big, so break up how many partitions we have
mailing_list_posts_mbox_ids = mailing_list_posts_mbox_ids.map(random_key).repartition(2000).map(de_key)
mailing_list_posts_rdd = mailing_list_posts_mbox_ids.flatMap(lambda args: fetch_and_process_mbox_records(*args))
mailing_list_posts_rdd.persist(StorageLevel.MEMORY_AND_DISK)

PythonRDD[216] at RDD at PythonRDD.scala:48

In [61]:
schema = StructType([
    StructField("project_name",StringType()),
    StructField("box_type",StringType()), # dev or user
    StructField("mbox_id",StringType()),
    StructField("backend_name",StringType()),
    StructField("backend_version",StringType()),
    StructField("category",StringType()),
    StructField("data", MapType(StringType(),StringType())), # The "important" bits
    StructField("origin",StringType()),
    StructField("perceval_version",StringType()),
    StructField("tag",StringType()),
    StructField("timestamp",DoubleType()),
    StructField("updated_on",DoubleType()),
    StructField("uuid",StringType())])
mailing_list_posts_mbox_df_raw = mailing_list_posts_rdd.toDF(schema=schema)
mailing_list_posts_mbox_df_raw.persist(StorageLevel.MEMORY_AND_DISK)
mailing_list_posts_mbox_df_raw.alias("Mailing list perceival information - no post processing")

DataFrame[project_name: string, box_type: string, mbox_id: string, backend_name: string, backend_version: string, category: string, data: map<string,string>, origin: string, perceval_version: string, tag: string, timestamp: double, updated_on: double, uuid: string]

In [62]:
mailing_list_posts_mbox_df_raw = non_blocking_df_save_or_load(
    mailing_list_posts_mbox_df_raw,
    "{0}mailing_list_info_4".format(fs_prefix))

Reusing


In [63]:
records = mailing_list_posts_mbox_df_raw.take(5)

In [64]:
records[0]

Row(project_name='juddi', box_type='dev', mbox_id='201302', backend_name='MBox', backend_version='0.10.2', category='message', data={'Reply-To': 'dev@juddi.apache.org', 'X-Original-To': 'apmail-juddi-dev-archive@www.apache.org', 'Return-Path': '<dev-return-5466-apmail-juddi-dev-archive=juddi.apache.org@juddi.apache.org>', 'List-Post': '<mailto:dev@juddi.apache.org>', 'Received': 'from arcas.apache.org (HELO arcas.apache.org) (140.211.11.28)\n    by apache.org (qpsmtpd/0.29) with ESMTP; Thu, 14 Feb 2013 13:38:13 +0000', 'Message-ID': '<JIRA.12632374.1360849017274.288795.1360849093042@arcas>', 'Content-Transfer-Encoding': '7bit', 'From': '"Steve Webb (JIRA)" <juddi-dev@ws.apache.org>', 'body': '{plain=Steve Webb created JUDDI-553:\n--------------------------------\n\n             Summary: TModelDetail not deleted when TModel is deleted\n                 Key: JUDDI-553\n                 URL: https://issues.apache.org/jira/browse/JUDDI-553\n             Project: jUDDI\n          Issue Type

In [65]:
mailing_list_posts_mbox_df = mailing_list_posts_mbox_df_raw.select(
    "*",
    mailing_list_posts_mbox_df_raw.data.getField("From").alias("from"),
    extract_email(mailing_list_posts_mbox_df_raw.data.getField("From")).alias("from_processed_email"),
    mailing_list_posts_mbox_df_raw.data.getField("body").alias("body"),
    mailing_list_posts_mbox_df_raw.data.getField("Message-ID").alias("message_id"),
    mailing_list_posts_mbox_df_raw.data.getField("In-Reply-To").alias("in_reply_to"),
    mailing_list_posts_mbox_df_raw.data.getField("Content-Language").alias("content_language")
    )

In [66]:
mailing_list_posts_mbox_df_saved = non_blocking_df_save_or_load(
    mailing_list_posts_mbox_df,
    "{0}/processed_mbox_data_3".format(fs_prefix))

Reusing


Start using some of the lazily created DFs to compute the sample

In [67]:
authors_grouped_by_id_saved.count()

15440

In [68]:
authors_grouped_by_id_saved.show()

+------------+--------------------+--------------------+--------------------+---------------+-------------+
|project_name|       new_unique_id|              emails|              Author|github_username|latest_commit|
+------------+--------------------+--------------------+--------------------+---------------+-------------+
|    accumulo|  <dhutchis@mit.edu>|[<dhutchis@mit.edu>]|Dylan Hutchison <...|               |   2015-01-02|
|    accumulo|<mario.pastorelli...|[<mario.pastorell...|Mario Pastorelli ...|               |   2015-12-30|
|    activemq|<czobrisky@gmail....|[<czobrisky@gmail...|Chad Zobrisky <cz...|               |   2014-12-31|
|       arrow|<adeneche@dremio....|[<adeneche@dremio...|adeneche <adenech...|               |   2017-01-06|
|       atlas|<amestry@apache.org>|[<amestry@apache....|Ashutosh Mestry <...|               |   2017-01-06|
|       atlas|<venkat@hortonwor...|[<venkat@hortonwo...|Venkat Ranganatha...|               |   2015-01-02|
|      aurora|<ndonatucci@me

In [69]:
num_authors_by_project = authors_grouped_by_id_saved.groupBy("project_name").agg(F.count("Author").alias("author_count"))
num_authors_by_project.cache()
num_authors_by_project.show()

+-------------+------------+
| project_name|author_count|
+-------------+------------+
|         lucy|          14|
|      vxquery|          16|
|    chemistry|          12|
|       roller|          19|
|        geode|         136|
|       falcon|          55|
|trafficserver|         307|
|          tez|          26|
|       pdfbox|          21|
|        httpd|         113|
|   carbondata|         135|
|        celix|          13|
|       wicket|          98|
|     accumulo|          95|
|        twill|          38|
|   servicemix|          19|
|     clerezza|          17|
|      couchdb|         169|
|       bigtop|         139|
|     marmotta|          25|
+-------------+------------+
only showing top 20 rows



Compute the sample %s for each project so we can get reasonable confidence bounds for sampling.
Looking at http://veekaybee.github.io/2015/08/04/how-big-of-a-sample-size-do-you-need/

In [70]:
# I think is Cochran's formula scaled for small datasets
@F.udf(IntegerType())
def compute_num_required_sample_1(pop_size):
    import numpy as np
    import scipy.stats
    import math
    e = 0.05
    Z = 1.64 # 90%, 95%: 1.96
    p = 0.5
    N = pop_size
    # CALC SAMPLE SIZE
    n_0 = ((Z**2) * p * (1-p)) / (e**2)
    # ADJUST SAMPLE SIZE FOR FINITE POPULATION
    n = n_0 / (1 + ((n_0 - 1) / float(N)) )
    target = int(math.ceil(n))
    # Compute a fall back size
    fall_back_size = min(3, pop_size)
    return max(fall_back_size, target) # THE SAMPLE SIZE

In [71]:
# Number 2: https://en.wikipedia.org/wiki/Sample_size_determination#Estimation
def walds_method():
    return 1/(0.05**2) # +- 5%
walds_method()

399.99999999999994

In [72]:
raw_sample_sizes = num_authors_by_project.withColumn(
    "sample_size_1",
    compute_num_required_sample_1("author_count")).persist()

In [73]:
sample_sizes = non_blocking_df_save_or_load(raw_sample_sizes, "{0}/sample_sizes_10".format(fs_prefix))

Reusing


In [74]:
sample_sizes.show()

+------------+------------+-------------+
|project_name|author_count|sample_size_1|
+------------+------------+-------------+
|spamassassin|          39|           35|
|        oodt|          41|           36|
|     cayenne|          45|           39|
|     streams|          25|           23|
|  jackrabbit|          37|           33|
|  manifoldcf|          19|           18|
|        drat|          29|           27|
|      giraph|          37|           33|
|     cordova|           1|            1|
|      thrift|         237|          127|
|      tomcat|          34|           31|
|     syncope|          31|           28|
|     calcite|         159|          101|
|      impala|         131|           89|
|       nutch|          72|           57|
|    activemq|          90|           68|
|  deltaspike|          56|           47|
|      allura|          84|           65|
|openmeetings|           7|            7|
|       tomee|          47|           41|
+------------+------------+-------

In [75]:
sample_sizes.groupby().agg(F.sum("sample_size_1")).show()

+------------------+
|sum(sample_size_1)|
+------------------+
|              9117|
+------------------+



So this is a bit high to do on a shoestring budget with sampling, but what about if we limit to folks who have recently participated & got rid of projects with limited or no recent participation.

In [76]:
authors_grouped_by_id_saved.schema

StructType(List(StructField(project_name,StringType,true),StructField(new_unique_id,StringType,true),StructField(emails,ArrayType(StringType,true),true),StructField(Author,StringType,true),StructField(github_username,StringType,true),StructField(latest_commit,DateType,true)))

In [77]:
active_distinct_authors_latest_commit = authors_grouped_by_id_saved.filter(
    (F.date_sub(F.current_date(), 365)) < authors_grouped_by_id_saved.latest_commit)

In [78]:
active_distinct_authors_latest_commit.schema

StructType(List(StructField(project_name,StringType,true),StructField(new_unique_id,StringType,true),StructField(emails,ArrayType(StringType,true),true),StructField(Author,StringType,true),StructField(github_username,StringType,true),StructField(latest_commit,DateType,true)))

In [79]:
active_distinct_authors_latest_commit.show()
print(active_distinct_authors_latest_commit.count())
active_distinct_authors_latest_commit.take(5)

+------------+--------------------+--------------------+--------------------+---------------+-------------+
|project_name|       new_unique_id|              emails|              Author|github_username|latest_commit|
+------------+--------------------+--------------------+--------------------+---------------+-------------+
|        beam|<p.kaczmarczyk@oc...|[<p.kaczmarczyk@o...|Pawel Kaczmarczyk...|               |   2018-01-02|
|   cassandra|<MWeiser@ardmoref...|[<MWeiser@ardmore...|Matthias Weiser <...|               |   2018-01-02|
|         cxf|<butkovic@gmail.com>|[<butkovic@gmail....|Peter Butkovic <b...|               |   2017-12-31|
|         cxf|<simon.marti@inve...|[<simon.marti@inv...|Simon Marti <simo...|               |   2017-12-31|
|       eagle| <yonzhang@ebay.com>|[<yonzhang@ebay.c...|yonzhang <yonzhan...|               |   2018-01-02|
|        hive| <ganu.ec@gmail.com>|[<ganu.ec@gmail.c...|Ganesha Shreedhar...|               |   2018-01-05|
|       karaf| <ancosen@gmai

[Row(project_name='beam', new_unique_id='<p.kaczmarczyk@ocado.com>', emails=['<p.kaczmarczyk@ocado.com>'], Author='Pawel Kaczmarczyk <p.kaczmarczyk@ocado.com>', github_username='', latest_commit=datetime.date(2018, 1, 2)),
 Row(project_name='cassandra', new_unique_id='<MWeiser@ardmorefinancial.com>', emails=['<MWeiser@ardmorefinancial.com>'], Author='Matthias Weiser <MWeiser@ardmorefinancial.com>', github_username='', latest_commit=datetime.date(2018, 1, 2)),
 Row(project_name='cxf', new_unique_id='<butkovic@gmail.com>', emails=['<butkovic@gmail.com>'], Author='Peter Butkovic <butkovic@gmail.com>', github_username='', latest_commit=datetime.date(2017, 12, 31)),
 Row(project_name='cxf', new_unique_id='<simon.marti@inventage.com>', emails=['<simon.marti@inventage.com>'], Author='Simon Marti <simon.marti@inventage.com>', github_username='', latest_commit=datetime.date(2017, 12, 31)),
 Row(project_name='eagle', new_unique_id='<yonzhang@ebay.com>', emails=['<yonzhang@ebay.com>'], Author='yo

In [80]:
active_num_authors_by_project = active_distinct_authors_latest_commit.groupBy("project_name").agg(F.count("Author").alias("author_count"))
active_num_authors_by_project.cache()
active_num_authors_by_project.show()

+-------------+------------+
| project_name|author_count|
+-------------+------------+
|         lucy|           2|
|       roller|           1|
|        geode|          47|
|       falcon|           4|
|trafficserver|          35|
|          tez|           6|
|       pdfbox|           4|
|        httpd|          14|
|   carbondata|          33|
|        celix|           6|
|     accumulo|          12|
|       wicket|          14|
|        twill|           2|
|      couchdb|          22|
|       bigtop|          12|
|     marmotta|           2|
|          vcl|           1|
|   freemarker|           4|
|       buildr|           2|
|        shiro|           2|
+-------------+------------+
only showing top 20 rows



In [81]:
active_raw_sample_sizes = active_num_authors_by_project.withColumn(
    "sample_size_1",
    compute_num_required_sample_1("author_count")).persist()

In [82]:
active_sample_sizes = non_blocking_df_save_or_load_csv(
    active_raw_sample_sizes,
    "{0}/active_sample_sizes_13".format(fs_prefix))

Reusing


In [83]:
filtered_active_sample_sizes = active_sample_sizes.filter(
    active_sample_sizes.sample_size_1 > 10).persist()

In [84]:
filtered_active_sample_sizes.groupby().agg(F.sum("sample_size_1")).show()

+------------------+
|sum(sample_size_1)|
+------------------+
|              1605|
+------------------+



That's probably ok, lets go ahead and compute the sample set for each project

In [85]:
sample_fractions = filtered_active_sample_sizes.withColumn(
    "sample_fraction",
    filtered_active_sample_sizes.sample_size_1 / filtered_active_sample_sizes.author_count)

In [86]:
local_sample_fractions = sample_fractions.select(sample_fractions.project_name, sample_fractions.sample_fraction).collect()

In [87]:
sampled_authors = active_distinct_authors_latest_commit.sampleBy(
    "project_name",
    fractions=dict(map(lambda r: (r[0], r[1]), local_sample_fractions)),
    seed=42)

In [88]:
sampled_authors_saved = non_blocking_df_save_or_load(
    sampled_authors, "{0}/sampled_authors_3".format(fs_prefix))

Reusing


In [89]:
sampled_authors_saved.show()

+------------+--------------------+--------------------+--------------------+---------------+-------------+
|project_name|       new_unique_id|              emails|              Author|github_username|latest_commit|
+------------+--------------------+--------------------+--------------------+---------------+-------------+
|      ambari|<adoroszlai@apach...|[<adoroszlai@apac...|Doroszlai, Attila...|               |   2018-01-05|
|      ambari|<kinow@users.nore...|[<kinow@users.nor...|Bruno P. Kinoshit...|               |   2017-12-31|
|       arrow|           JinHai-CN|[<haijin.chn@gmai...|Jin Hai <haijin.c...|      JinHai-CN|   2018-01-03|
|        beam|<jmarble@kochava....|[<jmarble@kochava...|Jacob Marble <jma...|               |   2018-01-05|
|        beam|<udim@users.norep...|[<udim@users.nore...|Udi Meiri (Ehud) ...|               |   2018-01-05|
|     calcite|<maryann.xue@gmai...|[<maryann.xue@gma...|maryannxue <marya...|               |   2018-01-04|
|       camel|             g

In [90]:
sampled_authors_saved.count()

1599

Join the sampled authors with the e-mails on the dev list and find the top 3 most recent responses

In [91]:
sampled_authors_saved.schema

StructType(List(StructField(project_name,StringType,true),StructField(new_unique_id,StringType,true),StructField(emails,ArrayType(StringType,true),true),StructField(Author,StringType,true),StructField(github_username,StringType,true),StructField(latest_commit,DateType,true)))

In [92]:
mailing_list_posts_mbox_df_saved.printSchema()

root
 |-- project_name: string (nullable = true)
 |-- box_type: string (nullable = true)
 |-- mbox_id: string (nullable = true)
 |-- backend_name: string (nullable = true)
 |-- backend_version: string (nullable = true)
 |-- category: string (nullable = true)
 |-- data: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- origin: string (nullable = true)
 |-- perceval_version: string (nullable = true)
 |-- tag: string (nullable = true)
 |-- timestamp: double (nullable = true)
 |-- updated_on: double (nullable = true)
 |-- uuid: string (nullable = true)
 |-- from: string (nullable = true)
 |-- from_processed_email: string (nullable = true)
 |-- body: string (nullable = true)
 |-- message_id: string (nullable = true)
 |-- in_reply_to: string (nullable = true)
 |-- content_language: string (nullable = true)



In [93]:
join_conditions = [
    sampled_authors_saved.project_name == mailing_list_posts_mbox_df_saved.project_name,
    F.expr("array_contains(emails, from_processed_email)")]
posts_by_sampled_authors = sampled_authors_saved.join(
    mailing_list_posts_mbox_df_saved,
    join_conditions).select(
    "message_id", "new_unique_id").alias("posts_by_sampled_authors")

In [94]:
posts_by_sampled_authors.show()

+--------------------+--------------------+
|          message_id|       new_unique_id|
+--------------------+--------------------+
|<CANDVwqg13PByzo_...|<ankitsinghal59@g...|
|<CANDVwqjRaTApW7G...|<ankitsinghal59@g...|
|<57465780.7010801...| <elserj@apache.org>|
|<CANDVwqgNEhpxw0y...|<ankitsinghal59@g...|
|<CALcavhvKkGNey-=...|<maryann.xue@gmai...|
|<CALcavhv32a+RvJV...|<maryann.xue@gmai...|
|<1391490711.89825...|           lhofhansl|
|<1391493045.37071...|           lhofhansl|
|<1391667527.54276...|           lhofhansl|
|<1391716442.94641...|           lhofhansl|
|<1391931449.79690...|           lhofhansl|
|<1391931663.53663...|           lhofhansl|
|<1392077000.11292...|           lhofhansl|
|<1392587082.23386...|           lhofhansl|
|<1392599989.30313...|           lhofhansl|
|<1392601705.67962...|           lhofhansl|
|<CAN=qJ2A4+F926bP...|            jmahonin|
|<CAN=qJ2C7j=TfF7_...|            jmahonin|
|<CAN=qJ2DseRA3bOd...|            jmahonin|
|<CAN=qJ2DUJ_73gBz...|          

In [95]:
mailing_list_posts_in_reply_to = mailing_list_posts_mbox_df_saved.filter(
    mailing_list_posts_mbox_df_saved.in_reply_to.isNotNull()).alias("mailing_list_posts_in_reply_to")

In [96]:
posts_with_replies = posts_by_sampled_authors.join(
    mailing_list_posts_in_reply_to,
    [F.col("mailing_list_posts_in_reply_to.in_reply_to") == posts_by_sampled_authors.message_id],
    "inner")

In [97]:
posts_in_response_to_user = posts_with_replies.select(
    posts_with_replies.new_unique_id,
    posts_with_replies.timestamp,
    posts_with_replies.project_name,
    posts_with_replies.body)

In [98]:
common_pronouns = ["they", "ze", "he", "she", "her", "his", "their"]

In [99]:
from sparklingml.feature.python_pipelines import SpacyTokenizeTransformer
tokenizer = SpacyTokenizeTransformer(inputCol="body", outputCol="body_tokens")

Using backing jar /sparklingml/sparklingml/../target/scala-2.11/sparklingml-assembly-0.0.1-SNAPSHOT.jar


In [100]:
posts_in_response_to_user_tokenized = tokenizer.transform(posts_in_response_to_user)

In [102]:
# Hack until I fix sparklingml
def mini_tokenize(doc):
    return doc.lower().split(" ")

mini_tokenize_udf = UserDefinedFunction(mini_tokenize, StringType(), "mini_tokenize_udf")


posts_in_response_to_user_tokenized = posts_in_response_to_user.withColumn(
    "body_tokens",
    mini_tokenize_udf(posts_in_response_to_user.body))

In [105]:
def contains_pronoun(tokens):
    return any(pronoun in tokens for pronoun in common_pronouns)

In [112]:
contains_pronoun_udf = UserDefinedFunction(contains_pronoun, BooleanType(), "contains_pronoun")

In [117]:
posts_in_response_to_user_with_pronouns = posts_in_response_to_user_tokenized.filter(
    contains_pronoun_udf(posts_in_response_to_user_tokenized.body_tokens))

In [118]:
posts_in_response_to_user_grouped = posts_in_response_to_user_with_pronouns.orderBy(
    posts_in_response_to_user.timestamp).groupBy(posts_in_response_to_user.new_unique_id)

In [125]:
posts_in_response_to_user_collected = posts_in_response_to_user_grouped.agg(
    F.collect_list(posts_in_response_to_user_with_pronouns.body).alias("emails"))

In [126]:
posts_in_response_to_user_collected_saved = non_blocking_df_save_or_load(
    posts_in_response_to_user_collected,
    "{0}/posts_by_user".format(fs_prefix))

Saving


Create a sample for analysis

Attempt to infer Gender off of name. This has problems, see https://ironholds.org/names-gender/ for a discussion on why this is problematic, but if it matches our statistical samples from above it can augment our understanding of the data. However without doing this it's difficult to get much of a picture (see above where we attempt to gender from other sources, the hit rate leaves something to be desired)